In [205]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
from bs4 import BeautifulSoup

In [206]:
url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173-526911--,00.html'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [207]:
tables = soup.find_all('table')
#[len(a) for a in a[0].find_all('tr')]

In [208]:
rows = tables[0].find_all('tr')
for row in rows:
    if len(row) == 5:
        cells = row.find_all('td')
        if len(cells) == 2: # TODO = Should throw error if not
            print (cells[0].get_text(), cells[1].get_text())

Resident Confirmed Cases * 7582
Residents Recovered or Recovering  5736
Resident COVID-19 Deaths * 1986
Staff Confirmed Cases * 3697
Staff COVID-19 Deaths * 21


In [209]:
cells = rows[2].find_all('td')
[a.get_text() for a in cells]

['Resident COVID-19 Deaths *', '1986']

In [210]:
def fix_MI_values(val):
    if val == '--':
        return np.nan
    if type(val) == int:
        return val
    else :
        return int(val.replace('*', '').strip())

In [211]:
rows = tables[2].find_all('tr')
county="ERROR" # SHould throw exception and catch these, but for now
facilities = []
new_county = True
for row in rows:
    if len(row) == 11:
        cells = row.find_all('td')
        if new_county:
            headers = ['County'] + [cell.get_text() for cell in cells]
            headers[2] = 'Resident ' + headers[2]
            headers[3] = 'Resident ' + headers[3]
            headers[4] = 'Staff ' + headers[4]
            headers[5] = 'Staff ' + headers[5]
            #print(headers)
            new_county = False
        else:
            values = [cell.get_text() for cell in cells]
            #values = [values[0]] + [int(a.replace('*', '').replace('--', '-1').strip()) for a in values[1:]]
            values = [values[0]] + [fix_MI_values(a) for a in values[1:]]
            facilities.append(dict(zip(headers, [county] + values)))
            
    if len(row) == 7:
        cells = row.find_all('td')
        county=cells[0].get_text()
        new_county = True
        #print('County = ' + cells[0].get_text())

In [212]:
df = pd.DataFrame(facilities)

df[60:70]

,County,Facility Name,Resident Cumulative Confirmed Cases,Resident Cumulative COVID-19 Deaths,Staff Cumulative Confirmed Cases,Staff Cumulative COVID-19 Deaths
60,DICKINSON,Freeman Nursing & Rehab,0.0,0.0,3.0,0.0
61,DICKINSON,Manorcare Nursing and Rehab Center,0.0,0.0,0.0,0.0
62,EATON,Dimondale Nursing Care Center,0.0,0.0,0.0,0.0
63,EATON,Eaton County Health and Rehabilitation,1.0,0.0,6.0,0.0
64,EATON,Medilodge Of Lansing,0.0,0.0,0.0,0.0
65,EATON,Regency at Lansing West,4.0,2.0,4.0,0.0
66,EMMET,Bay Bluffs - Emmet Co. Medical Care,0.0,0.0,0.0,0.0
67,EMMET,Hiland Cottage,NaN,NaN,NaN,NaN
68,EMMET,The Villa at the Bay,0.0,0.0,0.0,0.0
69,GENESEE,Argentine Care Center,0.0,0.0,2.0,0.0


In [225]:
df = pd.DataFrame(facilities)
df_county = df.groupby(by=['County']).sum()
#TODO - add date to filename
#TODO - get date from scaping web site
df_county.sort_values('Resident Cumulative Confirmed Cases', ascending=False).to_csv('Reporting_data/MI_county_stats.csv')

In [226]:
df.sort_values('Resident Cumulative Confirmed Cases', ascending=False).to_csv('Reporting_data/MI_2020-07-22_LTC_data.csv', index=False)

In [121]:
a='2*'
int(a.replace('*', '').replace('--', '-1').strip())

2